<a href="https://colab.research.google.com/github/MaximeGloesener/HandsOnAI-Challenge1/blob/master/data_analyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Hardware Informations (GPU)**

In [ ]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

In [ ]:
!pip install ImageHash

# **2. Importation of librairies**

In [ ]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.models import Model, load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input #224*224
from keras.applications.xception import Xception, preprocess_input, decode_predictions #299*299
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions #224*224
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Flatten, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance
from google.colab import files
from tqdm import tqdm
import imagehash
print("Tensorflow version: "+tf.__version__)
print("Keras version: " + tf.keras.__version__)

#**3. Download of training datasets "FIRE_DATABASE_X"**

In [ ]:
bases_path_after="bases"
test="test_data"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)
if not os.path.exists(test):
  os.makedirs(test)

In [ ]:
!unzip -q archive\ \(1\).zip

In [ ]:
!rm -rf FIRE_DATABASE_1.tar
!rm -rf sample_data
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
!tar xf FIRE_DATABASE_1.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_1.tar

In [ ]:
!rm -rf FIRE_DATABASE_2.tar
!rm -rf sample_data
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
!tar xf FIRE_DATABASE_2.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_2.tar

In [ ]:
!rm -rf FIRE_DATABASE_3.tar
!rm -rf sample_data
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_3.tar

In [ ]:
len(os.listdir("/content/bases/FIRE_DATABASE_3/fire"))+len(os.listdir("/content/bases/FIRE_DATABASE_3/no_fire"))+len(os.listdir("/content/bases/FIRE_DATABASE_3/start_fire"))

In [ ]:
# Données de test
!rm -rf sample_data
!wget --no-check-certificate http://195.154.53.219/downloads/test.tar
! tar xf test.tar -C 'test_data' --one-top-level
! rm test.tar

Important de tester les doublons en utilisant un hash cryptographique qui comparer les images pixels par pixels. Avec un hash robuste, on trouve des faux doublons. Le hash robuste permet de détecter les doublons si resize/légère modifiction mais ce n'est pas le cas ici dans les datasets. 

In [ ]:
def read_image(file_name):
  """
  Fonction qui prend en entrée une path d'image et qui return RGB (utile pour plot)
  """
  img = cv2.imread(file_name, 3)
  b,g,r = cv2.split(img)
  rgb_image = cv2.merge([r,g,b])
  return rgb_image

def plot(images, noms):
  f, axarr = plt.subplots(1,len(images))
  for i in range(len(images)):
    axarr[i].imshow(images[i])
    axarr[i].title.set_text(noms[i])


In [ ]:
# Analyse des données
# On sait que dans les datasets, il y a parfois plusieurs fois la même image
# But : analyser chaque dataset et trouver le nombre d'images en doublons
def analyse_dataset(folder_name, affichage = False):
  """
  Fonction qui prend en entrée le directory d'un dataset et qui va chercher les images qui sont présentes plusieurs fois pour ce même dataset
  Affichage = True si on veut plot les images qui sont en doubles et leur nom
  Return: - le nombre de doublons dans un dataset
          - le pourcentage de doublons
  """
  img_hashes = dict()
  total = 0
  doublons = 0

  for dir in os.listdir(folder_name):
    for image in os.listdir(os.path.join(folder_name, dir)):
      total += 1
      image = os.path.join(os.getcwd(), folder_name, dir, image)
      hash = imagehash.dhash(Image.open(image))
      if hash in img_hashes:
        doublons += 1
        #print(f'{image} doublons de {img_hashes[hash]}')
        if affichage:
          i = read_image(image) 
          x = read_image(img_hashes[hash])
          plot([x,i],[image.split("/")[-1], img_hashes[hash].split("/")[-1]])
      else:
        img_hashes[hash] = image

  return doublons, doublons/total*100

d1, p1 = analyse_dataset("/content/bases/FIRE_DATABASE_1/")
d2, p2 = analyse_dataset("/content/bases/FIRE_DATABASE_2/")
d3, p3 = analyse_dataset("/content/bases/FIRE_DATABASE_3/", affichage = True)
print('DATASET 1 ')
print(f'Il y a {d1} doublons dans le dataset = {p1}% des données')
print('DATASET 2 ')
print(f'Il y a {d2} doublons dans le dataset = {p2}% des données')
print('DATASET 3 ')
print(f'Il y a {d3} doublons dans le dataset = {p3}% des données')


In [ ]:
# Créer un seul dataset à partir des 3 en ne prenant en compte que des images uniques (supprimer tous les doublons)
def make_dataset(base_directory):
  """
  Fonction qui va concaténer les 3 datasets de départ et créer un seul dataset sans doublons
  Return les hashs des images déjà présentes dans le dataset -> utile lors de la phase data augmentation pour 
  ne pas rajouter des images qui sont déjà présentes dans la jeu de données 
  """
  !rm -rf all_data
  directory = 'all_data'
  directory_path = os.path.join(os.getcwd(), directory)
  # créer un nouveau directory all_data s'il n'existe pas déjà
  if not os.path.exists(directory_path):
    os.mkdir(directory_path)
    os.mkdir(os.path.join(directory_path, "fire"))
    os.mkdir(os.path.join(directory_path, "no_fire"))
    os.mkdir(os.path.join(directory_path, "start_fire"))

  images_hash = set()
  for dir in os.listdir(base_directory):
    for dir2 in os.listdir(os.path.join(base_directory,dir)):
      for img in os.listdir(os.path.join(base_directory,dir,dir2)):
        path = os.path.join(os.getcwd(), base_directory, dir, dir2, img)
        hash = imagehash.dhash(Image.open(path))
        if hash not in images_hash:
          images_hash.add(hash)
          cv2.imwrite(os.path.join(directory_path, dir2, img), cv2.imread(path))
  return images_hash 
hashes = make_dataset("bases")

In [ ]:
# Stocker les hashs déjà présent dans notre DB d'images pour ne pas par la suite, rajouter des images déjà présentes
with open('hashes.txt','w') as fout:
  for hash in hashes:
    fout.write(f"{hash}\n")


In [ ]:
analyse_dataset("/content/all_data", affichage=True)

In [ ]:
len(os.listdir("/content/all_data/fire"))+len(os.listdir("/content/all_data/no_fire"))+len(os.listdir("/content/all_data/start_fire"))

In [ ]:
# Analyse des images dans un directory
directory = 'bases/test/start_fire'

for index, img in enumerate(os.listdir(directory)):
  img = os.path.join(os.getcwd(), directory, img)
  fig = plt.figure()
  image = read_image(img)
  plt.imshow(image)
  plt.title(img)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cp -av "/content/all_data/" "/content/gdrive/MyDrive/Challenge1/"